In [ ]:
import os
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Function
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader

from model.dataset import SMPLRandomDataset
from model.SkinningNet import JointNet, WeightBindingNet, SkinningNet
from model.utils import JointLoss, WeightBindingLoss, getSkeletonOneRingIdx

from meshplot import plot
import igl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:%s"%device)

In [ ]:
batchSize = 16
trainDataset = SMPLRandomDataset(complexity = "all", 
                                 gender = "mixed", 
                                 dataSize = 5000, 
                                 vertexOrderRandom = True, 
                                 noise = 1, 
                                 rotate = False, 
                                 scale = True,
                                 translate = True,
                                 centre = False)
trainIter = DataLoader(trainDataset, num_workers=0, batch_size=batchSize, shuffle=True, drop_last=True)
allTrainNUM = len(trainDataset)

In [ ]:
net = JointNet()
net.load_state_dict(torch.load(".\stateDict\jointNet_pretrain_noise.pkl"))
net = net.to(device)

weightNet = WeightBindingNet(jointNum = 24)
weightNet.load_state_dict(torch.load(".\stateDict\weightNet_pretrain_noise.pkl"))
weightNet.to(device)
weightNet = weightNet.eval()

In [ ]:
lr = 0.00005
epochs = 100
opt = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = CosineAnnealingLR(opt, epochs, eta_min=0.1*lr)

weightLoss = WeightBindingLoss()
loss = JointLoss()

In [ ]:
beginTime = time.time()
net = net.train()

skeletonOneRingIdx = getSkeletonOneRingIdx("human")
skeletonOneRingIdx = torch.tensor(skeletonOneRingIdx).unsqueeze(0).repeat(batchSize, 1, 1).to(device).long()
for epoch in range(epochs):
    epochBegin = time.time()
    trainLoss = 0.0
    completedNum = 0
    batchNum = 0
    trainAcc = 0.0
    for V, facesOneRingIdx, rigW, joints in trainIter:
        V = V.to(device).float()
        facesOneRingIdx = facesOneRingIdx.to(device).long()
        rigW = rigW.to(device).float()
        joints = joints.to(device).float()
        opt.zero_grad()
        
        with torch.no_grad():
            preA = weightNet(V, facesOneRingIdx)
            sl, acc = weightLoss(preA, rigW.permute(0, 2, 1))
            W = (preA == preA.max(dim = 1, keepdim = True)[0]).float()
        preJ = net(V, W, facesOneRingIdx, skeletonOneRingIdx)
        l = loss(preJ, joints)
        l.backward()
        opt.step()
        
        batchLoss = float(l)
        batchAcc = float(acc)
        trainAcc += batchAcc
        trainLoss += batchLoss
        batchNum += 1
        completedNum += batchSize
        compltePer = completedNum/allTrainNUM
        leftTime = ((time.time() - epochBegin)/compltePer - (time.time() - epochBegin))/60
        sys.stdout.write('\r 训练%i / %i, 关节点损失: %f, 准确率：%0.2f%%, 当前进度: %0.2f%%, 预计剩余%d分钟'%(completedNum, allTrainNUM, batchLoss, batchAcc*100, compltePer * 100, leftTime))
        sys.stdout.flush()
        
    torch.save(net.state_dict(), ".\stateDict\epochs\jointNet_%d.pkl"%epoch)
    print("\n")
    print("epoch%d, 关节点损失: %f, 准确率：%0.2f%%, 用时：%0.2f minuetes"%(epoch+1, trainLoss/batchNum, trainAcc/batchNum*100, (time.time() - beginTime) / 60 ))
    print("----------------------------------")
    scheduler.step()

In [ ]:
torch.save(net.state_dict(), ".\stateDict\jointNet_finetune_noise.pkl")

In [ ]:
# jointNet = JointNet()
# jointNet.load_state_dict(torch.load(".\stateDict\jointNet_finetune_noise.pkl"))

# weightNet = WeightBindingNet(jointNum = 24)
# weightNet.load_state_dict(torch.load(".\stateDict\weightNet_pretrain_noise.pkl"))

net = SkinningNet(net, weightNet)
net = net.to(device)

torch.save(net.state_dict(), ".\stateDict\skinningNet_finetune_noise.pkl")